In [9]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'

import pandas as pd
import datasets
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from sklearn.dummy import DummyClassifier
from collections import Counter
import os
from tqdm.auto import tqdm, trange
from glob import glob 
from collections import defaultdict, Counter
from os.path import basename

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

tqdm.pandas()

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)
device = "cuda:0"

# Talkdown Imbalanced

In [12]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'

long_train_df = pd.read_csv(peft_outdir + 'talkdown.train.labeled.csv')

long_test_df = pd.read_csv(peft_outdir + 'talkdown.test.labeled.csv')

long_train_df.head()

,reddit_reply_id,text1,text2,relationship,label,answer,predicted
0,dblfraj,Rate whether it is inappropriate for message A...,Is inappropriate for a an employee in large co...,employee_in_large_company,True,yes,yes
1,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",acquaintance,True,yes,yes
2,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",coworker,True,yes,yes
3,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a mentor to say ""Not co...",mentor,True,yes,yes
4,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",grandchild,True,yes,yes


In [4]:
long_test_df.head()

,reddit_reply_id,text1,text2,relationship,label,answer,predicted
0,c0jxv1l,Rate whether it is inappropriate for message A...,Rate whether it is inappropriate for message A...,employee_in_large_company,False,yes,yes
1,c0jxv1l,Rate whether it is inappropriate for message A...,Rate whether it is inappropriate for message A...,acquaintance,False,yes,yes
2,c0jxv1l,Rate whether it is inappropriate for message A...,Rate whether it is inappropriate for message A...,coworker,False,yes,yes
3,c0jxv1l,Rate whether it is inappropriate for message A...,Rate whether it is inappropriate for message A...,mentor,False,yes,yes
4,c0jxv1l,Rate whether it is inappropriate for message A...,Rate whether it is inappropriate for message A...,grandchild,False,yes,yes


In [10]:
Counter(long_train_df.label)

Counter({True: 127596, False: 382788})

In [5]:
rels = list(set(long_train_df.relationship))

def to_long(sdf):
    label = list(set(sdf.label))[0]
    d = {}
    for row in sdf.itertuples():
        d[row.relationship] = 1 if row.predicted == 'yes' else 0

    out = { 'label': 1 if label == True else 0 }
    for r in rels:
        out[r] = d[r]
    return pd.Series(out)

train_df = long_train_df.groupby('reddit_reply_id').progress_apply(to_long).reset_index()
test_df = long_test_df.groupby('reddit_reply_id').progress_apply(to_long).reset_index()

train_df.head()

  0%|          | 0/10416 [00:00<?, ?it/s]

,reddit_reply_id,label,employee_in_large_company,mentor,friend,uncle_aunt,doctor,rival,ex_dating,competitor,...,mentee,divorcee,best_friend,fan,club_member,sibling,landlord,grandchild,acquaintance,complete_stranger
0,c031fjj,1,1,1,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
1,c031now,1,1,1,0,1,1,1,0,1,...,1,0,0,1,1,0,1,1,1,1
2,c039dew,1,1,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
3,c039odr,1,1,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
4,c03ce97,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1


In [6]:
test_df.head()

,reddit_reply_id,label,employee_in_large_company,mentor,friend,uncle_aunt,doctor,rival,ex_dating,competitor,...,mentee,divorcee,best_friend,fan,club_member,sibling,landlord,grandchild,acquaintance,complete_stranger
0,c03txew,1,1,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
1,c047pok,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,c04i9i2,0,1,1,0,1,1,0,0,0,...,1,0,0,1,0,0,1,1,1,1
3,c06slc4,1,1,1,0,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
4,c0ar4of,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1


In [13]:
Counter(train_df.label)

Counter({1: 2604, 0: 7812})

In [11]:
clf = LogisticRegression(solver='lbfgs', max_iter=400, random_state=42)
#clf = RandomForestClassifier()

X = np.array(train_df.iloc[:,2:])
y = np.array(train_df.iloc[:,1])

clf.fit(X, y)

if True:
    X_test = np.array(test_df.iloc[:,2:])
    y_test = np.array(test_df.iloc[:,1])

    y_pred = clf.predict(X_test)
    
    print(f1_score(y_test, y_pred, average='macro'))

0.6236633311564904


# TalkDown Balanced

In [21]:
bal_long_train_df = pd.read_csv(peft_outdir + 'talkdown.train.balanced.labeled.csv')

bal_long_test_df = pd.read_csv(peft_outdir + 'talkdown.test.balanced.labeled.csv')

bal_long_train_df.head()

,reddit_reply_id,text1,text2,relationship,label,answer,predicted
0,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",grandchild,True,yes,yes
1,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",old_friend,True,yes,no
2,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a fan to say ""Not conde...",fan,True,yes,yes
3,dblfraj,Rate whether it is inappropriate for message A...,Is inappropriate for a an uncle or aunt to say...,uncle_aunt,True,yes,yes
4,dblfraj,Rate whether it is inappropriate for message A...,"Is inappropriate for a a person to say ""Not co...",classmate,True,yes,yes


In [22]:
def to_long(sdf):
    label = list(set(sdf.label))[0]
    d = {}
    for row in sdf.itertuples():
        d[row.relationship] = 1 if row.predicted == 'yes' else 0

    out = { 'label': 1 if label == True else 0 }
    for r in rels:
        out[r] = d[r]
    return pd.Series(out)

bal_train_df = bal_long_train_df.groupby('reddit_reply_id').progress_apply(to_long).reset_index()
bal_test_df = bal_long_test_df.groupby('reddit_reply_id').progress_apply(to_long).reset_index()

bal_train_df.head()

  0%|          | 0/5208 [00:00<?, ?it/s]

  0%|          | 0/652 [00:00<?, ?it/s]

,reddit_reply_id,label,employee_in_large_company,mentor,friend,uncle_aunt,doctor,rival,ex_dating,competitor,...,mentee,divorcee,best_friend,fan,club_member,sibling,landlord,grandchild,acquaintance,complete_stranger
0,c02a7xc,0,1,1,0,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
1,c02l31x,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
2,c031fjj,1,1,1,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
3,c031now,1,1,1,0,1,1,1,0,1,...,1,0,0,1,1,0,1,1,1,1
4,c0375ha,0,1,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1


In [23]:
clf = LogisticRegression(solver='lbfgs', max_iter=400, random_state=42)
#clf = RandomForestClassifier()

X = np.array(bal_train_df.iloc[:,2:])
y = np.array(bal_train_df.iloc[:,1])

clf.fit(X, y)

if True:
    X_test = np.array(bal_test_df.iloc[:,2:])
    y_test = np.array(bal_test_df.iloc[:,1])

    y_pred = clf.predict(X_test)
    
    print(f1_score(y_test, y_pred, average='macro'))

0.7078853046594982
